In [ ]:
import os
import sys
import json

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')
from fgc_support_retri.train import *

I0214 03:24:39.152928 139941226334016 file_utils.py:39] PyTorch version 1.1.0 available.


In [ ]:
train_SynSERModel(10, 12, "20200214_syn_all", "syn-all")

I0214 03:24:40.637109 139941226334016 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.3767c74c8ed285531d04153fe84a0791672aff52f7249b27df341dbce09b8305
I0214 03:24:40.641416 139941226334016 configuration_utils.py:169] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attentio

100
no gold supporting evidence
{'QID': 'D001Q11', 'QTYPE': '申论', 'QTEXT': '蘇東坡為何被後人認為是文學藝術史上的通才?', 'SENTS': [{'text': '苏东坡为何被后人认为是文学艺术史上的通才?', 'start': 0, 'end': 21}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Event', 'AMODE': 'Single-Span-Extraction', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '苏东坡为何被后人认为是文学艺术史上的通才?'}
no gold supporting evidence
{'QID': 'D006Q02', 'QTYPE': '申论', 'QTEXT': '「阿拉伯之春」運動中，走上街頭的民眾的訴求為何?', 'SENTS': [{'text': '「阿拉伯之春」运动中，', 'start': 0, 'end': 11}, {'text': '走上街头的民众的诉求为何?', 'start': 11, 'end': 24}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Object', 'AMODE': 'Single-Span-Extraction', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '「阿拉伯之春」运动中，走上街头的民众的诉求为何?'}
no gold supporting evidence
{'QID': 'D048Q09', 'QTYPE': '申论', 'QTEXT': '聊天機器人仰賴哪些方法讓回答愈來愈準確?', 'SENTS': [{'text': '聊天机器人仰赖哪些方法让回答愈来愈准确?', 'start': 0, 'end': 20}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}]

I0214 03:24:53.453822 139941226334016 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
  0%|          | 0/2219 [00:00<?, ?it/s]

start training ... 


/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
  0%|          | 0/213 [00:00<?, ?it/s]

...

  0%|          | 0/213 [00:00<?, ?it/s]

epoch 5 train_loss: 0.057


100%|██████████| 213/213 [03:36<00:00,  1.15s/it]


{'sp_em': 0.17370892018779344, 'sp_prec': 0.5520120724346077, 'sp_recall': 0.5228593784931814, 'sp_f1': 0.49271927863477166}
epoch 5 eval_recall: 0.523 eval_f1: 0.493


 29%|██▉       | 639/2219 [05:17<11:46,  2.24it/s]